In [1]:
import h5py
import numpy as np
import os
from pathlib import Path
main_dir = str(Path().resolve())

In [2]:
paths = np.load(os.path.join(main_dir,'demos','Bottle_model_on_policy_10_debug_1617312388085008300.npy'),allow_pickle=True)

In [3]:
test = ([info['tool_pos'] for info in paths[0]['env_infos']],[info['gaze_features'] for info in paths[0]['env_infos']])

train = (
        sum([[info['tool_pos'] for info in path['env_infos']] for path in paths[1:]],[]),
        sum([[info['gaze_features'] for info in path['env_infos']] for path in paths[1:]],[]),
)

In [5]:
gaze_features = train[1]+test[1]
tool_pos = train[0]+test[0]
print(len(gaze_features))

1919


In [ ]:
import h5py
dataset = h5py.File(os.path.join(main_dir,'gaze_capture','gaze_data',f'bottle_cont_gaze_data.h5'))
dataset.create_dataset('gaze_features', data=gaze_features)
dataset.create_dataset('tool_pos', data=tool_pos)


In [28]:
import torch as th
from torch import nn
from torch.nn import functional as F
from torch import optim

dropout = .5
model = nn.Sequential(*[
    nn.Linear(128,128),
    nn.AlphaDropout(dropout),
    nn.ReLU(),
    nn.Linear(128,128),
    nn.AlphaDropout(dropout),
    nn.ReLU(),
    nn.Linear(128,128),
    nn.AlphaDropout(dropout),
    nn.ReLU(),
    nn.Linear(128,3)
])
optimizer = optim.Adam(model.parameters(),lr=5e-4)

In [30]:
from tqdm import tqdm
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# %matplotlib qt

epochs = 1000
t = tqdm(range(epochs))
for epoch in t:
    model.train()
    for y,x in zip(np.array_split(train[0],10),np.array_split(train[1],10)):
        x = th.tensor(x)
        y = th.tensor(y).float()
        pred_y = model(x)
        loss = F.mse_loss(pred_y,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        nmse = th.var(pred_y-y)/th.var(pred_y)
        rmse = th.sqrt(loss)
        print_nmse = np.around(nmse.item(),3)
        print_rmse = np.around(rmse.item(),3)
    model.eval()
    test_x = th.tensor(test[1])
    test_y = th.tensor(test[0]).float()
    test_pred_y = model(test_x)
    test_loss = F.mse_loss(test_pred_y,test_y)
    nmse = th.var(test_pred_y-test_y)/th.var(test_pred_y)
    rmse = th.sqrt(test_loss)
    print_nmse = np.around(nmse.item(),3)
    print_mse = np.around(rmse.item(),3)
    t.set_description('test--'+"nmse: "+str(print_nmse)+" rmse:"+str(print_mse))
    
# fig = plt.figure()
# ax = Axes3D(fig)
# ax.scatter(*(pred_y.numpy().T),label="predicted target positions")
# ax.scatter(*(y.numpy().T),label="predicted target positions")
# plt.legend()
# plt.pause(0.05)
# plt.show()
#     plt.close()

test--nmse: 0.022 rmse:0.106: 100%|██████████| 1000/1000 [01:52<00:00,  8.90it/s]
